In [1]:
import pandas as pd
import numpy as np
import re
import uuid

# Load dataset
usa_red_wines = pd.read_csv('Vivino_USA_redwines.csv')

In [2]:
# Filter columns that has information on wine AVA and varietal
usa_red_wines_varietals = usa_red_wines.loc[:, usa_red_wines.filter(like='wineStyles').columns]

# Convert to dummy variable columns
for col in usa_red_wines_varietals:
    usa_red_wines_varietals.loc[:, col] = usa_red_wines_varietals.loc[:, col].map(lambda x: 0 if pd.isna(x) else 1)

# Combime them into one column
usa_red_wines_varietals.loc[:, 'region_and_varietal'] = usa_red_wines_varietals.idxmax(axis=1)

In [3]:
# Extract wine AVA location
usa_red_wines_varietals['region'] = usa_red_wines_varietals['region_and_varietal'].str.extract(r'(Alexander Valley|Knights Valley|Paso Robles|Russian River Valley|Santa Lucia Highlands|Santa Maria Valley|Sonoma Coast|Sonoma County|Sta. Rita Hills|Napa Valley|Oregon)')

# Classify remaining wines without specific AVA location as "California"
usa_red_wines_varietals.loc[usa_red_wines_varietals.loc[:, 'region'].isna(),'region'] = 'California'

# Group Sta. Rita Hills and Santa Lucia Highlands under Santa Barbara AVA
usa_red_wines_varietals.loc[usa_red_wines_varietals['region'].str.contains(r'(Sta. Rita Hills|Santa Lucia Highlands)'), 'region'] = 'Santa Barbara'

# Extract wine varietal
usa_red_wines_varietals['varietal'] = usa_red_wines_varietals['region_and_varietal'].str.extract(r'(Cabernet Sauvignon|Blend|Merlot|Pinot Noir|Zinfandel)')

/var/folders/rs/dwj3n8716t593gn_r4x21y0r0000gn/T/ipykernel_49410/4140735168.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  usa_red_wines_varietals.loc[usa_red_wines_varietals['region'].str.contains(r'(Sta. Rita Hills|Santa Lucia Highlands)'), 'region'] = 'Santa Barbara'


In [4]:
# Select rows to keep and concatenate with varietal and region dataframe
df_california_wines = usa_red_wines.loc[:, ['wineryName', 'name', 'year', 'acidity', 'intensity', 'sweetnes', 'tannin', 'price', 'ratingAverage', 'numberTasterReviews', 'totalRatings','flavours/0/type','flavours/1/type','flavours/2/type' ]]
df_california_wines = df_california_wines.merge(usa_red_wines_varietals.loc[:, ['region', 'varietal']], how='left', left_index=True, right_index=True)
df_california_wines['uid'] = df_california_wines.apply(lambda _: uuid.uuid4(), axis=1)
df_california_wines

,wineryName,name,year,acidity,intensity,sweetnes,tannin,price,ratingAverage,numberTasterReviews,totalRatings,flavours/0/type,flavours/1/type,flavours/2/type,region,varietal,uid
0,Paul Hobbs,Paul Hobbs Beckstoffer Las Piedras Vineyard Ca...,2015,3.371048,4.626113,1.805029,3.323693,346.95,4.8,68,86,oak,black_fruit,earth,Alexander Valley,Cabernet Sauvignon,d796b115-a127-4e9b-91cc-6701eecd0bd2
1,Bernardus,Bernardus Marinus 2013,2013,3.743659,4.761420,1.572299,3.692956,34.99,4.3,51,98,oak,earth,black_fruit,Alexander Valley,Cabernet Sauvignon,d0ee868d-74b7-42f8-ba58-318c62f48665
2,Beau Vigne,Beau Vigne Legacy Cabernet Sauvignon 2020,2020,3.343543,4.570415,1.643999,3.484988,59.99,4.2,88,69,oak,black_fruit,red_fruit,Napa Valley,Cabernet Sauvignon,155c23cb-037e-4a49-b2e0-0754ab5f0edd
3,DAOU,DAOU Reserve Cabernet Sauvignon 2021,2021,3.059963,4.795351,1.668042,3.384822,39.99,4.3,510,689,oak,black_fruit,earth,Paso Robles,Cabernet Sauvignon,e97694b3-9767-4802-a6ef-bdd06d7e4384
4,Paul Hobbs,Paul Hobbs Beckstoffer Las Piedras Vineyard Ca...,2015,3.371048,4.626113,1.805029,3.323693,346.95,4.8,68,86,oak,black_fruit,earth,Alexander Valley,Cabernet Sauvignon,44538efb-b191-4d47-80e0-2c7aec7c27fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Beau Vigne,Beau Vigne Romeo Cabernet Sauvignon 2021,2021,3.355670,4.587563,1.728072,3.344321,99.99,4.4,133,1227,oak,black_fruit,non_oak,Napa Valley,Cabernet Sauvignon,5262029a-460f-4918-b09a-a5075aef27c4
148,J Dusi,J Dusi Paper Street The Narrator 2021,2021,4.038571,4.721000,1.538543,3.542857,29.99,4.4,20,158,black_fruit,red_fruit,oak,California,Blend,92e242f5-1bc0-45c5-924d-ad1fdecf7fbe
149,Château St. Jean,Château St. Jean Cinq Cépages 2017,2017,3.532186,4.797547,1.654068,3.604700,69.99,4.4,140,327,oak,black_fruit,earth,California,Blend,96537fcf-d11a-41ed-9aa6-85fc55e80e6d
150,Sbragia,Sbragia Neal Vineyard Cabernet Sauvignon 2017,2017,3.492442,4.980658,1.131783,3.660969,59.99,4.4,15,119,oak,black_fruit,red_fruit,Napa Valley,Cabernet Sauvignon,22113c3f-3ef3-4ee4-8e6c-29e42ae24b16


In [40]:
# Get initial distribution of varietals
df_california_wines.groupby('varietal').agg({'varietal':'count'})

,varietal
varietal,
Blend,31
Cabernet Sauvignon,100
Merlot,1
Pinot Noir,14
Zinfandel,6


In [17]:
# Get initial distribution of regions
df1 = df_california_wines.groupby('region').agg({'region':'count'})
df2 = df1.rename(columns = {'region':'count'})
df2.sort_values('count', ascending = False)

,count
region,
Napa Valley,75
California,41
Alexander Valley,13
Paso Robles,8
Sonoma County,5
Santa Barbara,3
Sonoma Coast,3
Russian River Valley,2
Knights Valley,1


In [42]:
# Get initial distribution of prices
df_california_wines.loc[:, 'price'].describe()

count     152.000000
mean       84.197039
std       286.252441
min         8.000000
25%        29.990000
50%        43.490000
75%        60.740000
max      3495.000000
Name: price, dtype: float64

In [43]:
# Get distribution of prices
df_california_wines.loc[:, 'price'].sort_values()

71        8.00
73       13.99
78       14.99
74       14.99
85       14.99
        ...   
4       346.95
0       346.95
105     369.99
110     425.00
112    3495.00
Name: price, Length: 152, dtype: float64

In [44]:
# Clean dataset by removing outliers

# Remove wines with grape varietals that only has one observation (Merlot)
df_california_wines = df_california_wines.loc[(df_california_wines.loc[:, 'varietal'] != "Merlot"), :]

# Remove wines from regions that only has one observation (Knights Valley)
df_california_wines = df_california_wines.loc[(df_california_wines.loc[:, 'region'] != "Knights Valley"), :]

# Remove wines from regions that only has one observation (Santa Maria Valley)
df_california_wines = df_california_wines.loc[(df_california_wines.loc[:, 'region'] != "Santa Maria Valley"), :]

# Remove wines which have outlier prices (USD 3495)
df_california_wines = df_california_wines.loc[(df_california_wines.loc[:, 'price'] != 3495), :]

In [45]:
# Get new distribution of varietals
df_california_wines.groupby('varietal').agg({'varietal':'count'})

,varietal
varietal,
Blend,31
Cabernet Sauvignon,98
Pinot Noir,13
Zinfandel,6


In [46]:
# Get new distribution of regions
df_california_wines.groupby('region').agg({'region':'count'})

,region
region,
Alexander Valley,13
California,40
Napa Valley,74
Paso Robles,8
Russian River Valley,2
Santa Barbara,3
Sonoma Coast,3
Sonoma County,5


In [ ]:
# Get new distribution of prices
df_california_wines.loc[:, 'price'].describe()

In [ ]:
# Get all winery names in dataset
df_california_wines.wineryName.unique()

In [53]:
flavor_df = df_california_wines.rename(columns = {'flavours/0/type':'flavor_1', 'flavours/1/type':'flavor_2', 'flavours/2/type':'flavor_3'})

In [55]:
flavor_df.to_csv('cali_wine.csv')


In [ ]:
def get_flavours_column(df, flavour_num, num_sub_flavours):
    """
    Create a dataframe for flavour descriptors and mentions by user reviews.

    Input:
        df (pandas dataframe): wine dataframe
        flavour_num (int): flavour group
        num_sub_flavours (int): number of subflavour descriptors to extract
    
    Returns:
        df (pandas dataframe): wine dataframe of flavours
    """
    base_text = "flavours/{type_num}"
    col_name = base_text.format(type_num=str(flavour_num))
    cols = []
    cols.append(col_name + '/type')
    cols.append(col_name + '/mentions')

    df_sub_flavours = df.loc[:, df.filter(like=col_name+'/taste').columns].iloc[:, :int(num_sub_flavours)]
    df_sub_flavours.loc[:, col_name + '/num_sub_flavours'] = df.loc[:, df.filter(like=col_name+'/taste').columns].notna().sum(axis=1)
    df_flavours = df.loc[:, cols]
    df = df_flavours.merge(df_sub_flavours, how='left', left_index=True, right_index=True)

    return df

In [ ]:
# Select first three flavour groups to create favlour dataframes
flavour_dfs = []
for i in range(10):
    # Use only first three descriptors for subflavours
    flavour_dfs.append(get_flavours_column(usa_red_wines, i, 0))

# Concatenate flavours dataframes with main dataframe
df_flavours = pd.concat(flavour_dfs, axis=1)

# Join main dataframe with flavours dataframe
df_california_wines_with_flavours = df_california_wines.merge(df_flavours, how='inner', left_index=True, right_index=True)
df_california_wines_with_flavours

In [ ]:
df_california_wines

In [ ]:
# Extract main flavour tyype columns and number of mentions
data = {}
for j in range(10):
    data['uid'] = list(df_california_wines_with_flavours.loc[:, 'uid'])
    data[f'flavours/{j}/type'] = list(df_california_wines_with_flavours.loc[:, f'flavours/{j}/type'])
    data[f'flavours/{j}/mentions'] = list(df_california_wines_with_flavours.loc[:, f'flavours/{j}/mentions'] / df_california_wines_with_flavours.loc[:, 'numberTasterReviews'])

df = pd.DataFrame(data)

# Create an empty DataFrame to store aggregated data
aggregated_data = []

# Iterate over each row and aggregate flavors
for i in range(len(df)):
    row = df.iloc[i]
    row_data = {
        'uid': None,
        'oak': 0,
        'black_fruit': 0,
        'earth': 0,
        'red_fruit': 0,
        'spices': 0,
        'non_oak': 0,
        'microbio': 0,
        'floral': 0,
        'tree_fruit': 0,
        'citrus_fruit': 0,
        'dried_fruit': 0,
        'tropical_fruit': 0,
        'vegetal': 0,
    }
    for j in range(9):
        flavour_type = row[f'flavours/{j}/type']
        mentions = row[f'flavours/{j}/mentions']
        if flavour_type in row_data:
            row_data[flavour_type] += mentions
            row_data['uid'] = row['uid']

    aggregated_data.append(row_data)

# Create DataFrame from aggregated data
aggregated_df = pd.DataFrame(aggregated_data)

aggregated_df

In [ ]:
# Merge dataframe with mention counts for each flavour type to main dataframe
df_california_wines = df_california_wines.merge(aggregated_df, how='inner', on='uid')
df_california_wines = df_california_wines.set_index('uid')
df_california_wines

In [ ]:
df_california_wines.to_csv('cali_wine.csv')